# Shor's Algorithm
***

Shor's algorithm is possibly the crown jewel of quantum computing so far. The goal of this algorithm is to find the prime factors of some large number $N$ and the key speedup this algorithm provides over the classical counterpart is by executing the period-finding part using a quantum computer. This has huge implications for cryptography and generally cyber-security.


Shor's algorithm is composed of the following steps:
1. Choose a co-prime $a$, where $a\in [2,N-1]$ and the greatest common divisor of $a$ and $N$ is 1.
1. Find the order (periodicity) of $a$ modulo $N$, i.e. the smallest integer $r$ such that $a^r\text{mod} N=1$
1. Obtain the factor of $N$ by computing the greatest common divisor of $a^{r/2} \pm 1$ and $N$.

## Defining the U - gate
We will be working with modular arithemtic base 15. First we need to create the unitary gate that will perform the 7 mod 15 operation.

In [ ]:
N = 15
m = int(np.ceil(np.log2(N)))

U_qc = QuantumCircuit(m)
U_qc.x(range(m))
U_qc.swap(1, 2)
U_qc.swap(2, 3)
U_qc.swap(0, 3)

U = U_qc.to_gate()
U.name ='{}Mod{}'.format(7, N)

Next, we will need a way to repeatedly apply the U gate:


In [ ]:
#This function will return a ControlledGate object which repeats the action
# of U, 2^k times
def cU_multi(k):
    sys_register_size = 4
    circ = QuantumCircuit(sys_register_size)
    for _ in range(2**k):
        circ.append(U, range(sys_register_size))
    
    U_multi = circ.to_gate()
    U_multi.name = '7Mod15_[2^{}]'.format(k)
    
    cU_multi = U_multi.control()
    return cU_multi

In [ ]:
def shor_qpe(k):
    
    k = 12
    a = 7
    m : int = k - 4
    #Step 1. Begin a while loop until a nontrivial guess is found
    gcheck : bool = True
    while gcheck:
        
        #Step 2.1. Construct a QPE circuit with m phase count qubits
        #  to guess the phase phi = s/r using the predefined function cU_multi()
        qcirc = QuantumCircuit(k, m)
        qcirc.draw(output = "mpl")
        for i in range (m):
            qcirc.h(i)
            qcirc.x(8)
        for i in range (m):
            qcirc.append(cU_multi(i), [i] + list(range(m, k)))
        qcirc.barrier()
        qft_dagger(qcirc, m)
        qcirc.barrier()
        qcirc.measure(range(m), range(m))
        qcirc.draw(output = "mpl")
 

        #Step 2b. Run the QPE circuit with a single shot, record the results
        # and convert the estimated phase bitstring to decimal
                
        sim = Aer.get_backend('aer_simulator')
        shots = 1
        qcirc_counts = execute(qcirc, sim, shots=shots).result().get_counts()
        plot_histogram(qcirc_counts, figsize=(9,5))
        for i in qcirc_counts.values():
            a : int = (len(str(i)))
            decval : float = float(i) / pow(10, a)

        #Step 3. Use the Fraction object to find the guess for r
        r = (Fraction(decval).limit_denominator(15))

        #Step 4. Now that r has been found, use the builtin greatest common deonominator
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]

        #Step 5. For each guess in guesses, check if at least one is a non-trivial factor
        # i.e.  (guess != 1 or N) and (N % guess == 0)
        
        for guess in guesses:
            if (guess != 1 and guess != N) and ( N % guess != 0):  
                print(guesses)
                !(gcheck)
                break
            else:
                print("no non trivial guesses")
    #Step 6. If a nontrivial factor is found return the list 'guesses', otherwise
    # continue the while loop
  
        if gcheck == True:
            continue
        else:
            break
    print(guesses)
    return guesses